In [1]:
import database.summarize_rds

In [2]:
database.summarize_rds.

NameError: name 'summarize_rds' is not defined

In [5]:
# Limited to 4 requests/minute, otherwise need publish/subscribe
endpoint = 'https://api.entur.io/realtime/v1/rest/vm'
conn = connect_to_rds()
current_hour, current_epoch = get_epoch_and_cet_24hr()

# Call Entur SIRI API (returns XML)
response = requests.get(endpoint)
root = ElementTree.fromstring(response.content)
# root = ElementTree.parse('vm.xml').getroot() # For testing without hitting API

# Look at list of active vehicles from response
root_dict = xml_to_dict(root)
vehicle_statuses = root_dict['ServiceDelivery']['VehicleMonitoringDelivery']['VehicleActivity']
clean_active_trips(vehicle_statuses) # Modifies in-place by deleting elements to save memory

current_hour, current_epoch = get_epoch_and_cet_24hr()
args_str = upload_to_rds(vehicle_statuses, conn, current_epoch)
conn.close()